In [ ]:
from agents import Agent, Runner, RunContextWrapper, trace, handoff, function_tool, trace
from pydantic import BaseModel
from dotenv import load_dotenv
import random


load_dotenv(override=True)

class ManagerEscalation(BaseModel):
    issue: str
    ticket_number: str
    why: str

@function_tool
def create_ticket(issue: str):
    tkt_no = random.randint(1000, 9999)
    print('Creating ticket for issue: ', {issue})
    return f"Ticket created for issue: {issue} with ticket number: {tkt_no}"

def ticket_agent_handoff(ctx: RunContextWrapper[None], input: ManagerEscalation):
    print('Escalating issue to manager: ', input.issue)
    print('Ticket number: ', input.ticket_number)
    print('Why: ', input.why)
    return f"Escalated issue to manager: {input.issue} with ticket number: {input.ticket_number}"

ticket_agent = Agent(
    name="Ticket Agent",
    instructions="You resolve tickets and return the ticket number.",
    model="gpt-4o-mini",
    tools=[create_ticket],
)

customer_care_agent = Agent(
    name="Customer Care Agent",
    instructions="You assist customers with their issues and resolve tickets."
                 "If you can resolve the issue, escalate the issue and return the ticket number.",
    handoffs=[handoff(
        ticket_agent,
        on_handoff=ticket_agent_handoff,
        input_type=ManagerEscalation
        )],
    model="gpt-4o-mini",
)

runner = Runner()
result = await runner.run(customer_care_agent, "I want refund for my watch, but your system is not working.")
print(result.final_output)

Escalating issue to manager:  Refund request for a watch due to system malfunction
Ticket number:  N/A
Why:  The customer is unable to process the refund through the system.
Creating ticket for issue:  {'Refund request for a watch due to system malfunction.'}
I've created a ticket for your refund request regarding the watch. Your ticket number is **6859**. If you need further assistance, feel free to ask!
